# EDA BIOMETRICO

In [1]:
import pandas as pd
import datetime as dt
import numpy as np
import locale
import holidays

In [2]:
# --- Configurar los días festivos para Colombia para 2025:
co_holidays = holidays.CountryHoliday('CO', years=[2025])

In [3]:
ruta ="dataset/produccion10.xls"
df_prod = pd.read_excel(ruta)
año = 2025
mes = 10

WARNING *** file size (2299606) not 512 + multiple of sector size (512)


In [4]:
df_prod.sample(10)

,Tiempo,ID de Usuario,Nombre,Apellido,Número de tarjeta,Dispositivo,Punto del evento,Verificación,Estado,Evento,Notas
247,2025-10-31 06:35:21,NaN,NaN,NaN,NaN,ENTRADA,ENTRADA-1,Huella,ENTRADA-1 Entrada,Acceso denegado,NaN
5322,2025-10-07 12:00:02,1.070922e+09,ERIKA,UMAÑA,NaN,SALIDA,SALIDA-1,Huella,SALIDA-1 Entrada,Apertura con tarjeta de proximidad,NaN
5247,2025-10-07 13:25:19,8.009297e+07,80092967,NaN,6749358.0,ENTRADA,ENTRADA-1,Tarjeta,ENTRADA-1 Entrada,Apertura con tarjeta de proximidad,NaN
173,2025-10-31 11:11:09,7.904486e+07,ERNESTO,VALENCIA,NaN,SALIDA,SALIDA-1,Huella,SALIDA-1 Entrada,Acceso fuera de horario,NaN
5116,2025-10-08 06:37:11,1.019115e+09,PAOLA,BAUTISTA,NaN,ENTRADA,ENTRADA-1,Huella,ENTRADA-1 Entrada,Apertura con tarjeta de proximidad,NaN
2647,2025-10-21 11:59:36,1.072669e+09,CAROLINA,PINZON,NaN,ENTRADA,ENTRADA-1,Huella,ENTRADA-1 Entrada,Apertura con tarjeta de proximidad,NaN
4910,2025-10-08 16:10:58,1.072653e+09,MARIA ANGELICA,HERRERA,NaN,SALIDA,SALIDA-1,Huella,SALIDA-1 Entrada,Apertura con tarjeta de proximidad,NaN
5172,2025-10-07 16:31:40,5.141640e+06,ADRIANA,MARIN,NaN,SALIDA,SALIDA-1,Huella,SALIDA-1 Entrada,Apertura con tarjeta de proximidad,NaN
5394,2025-10-07 07:20:55,1.073255e+09,CRISTIAN,DUSSAN,NaN,logistica,logistica-1,Huella,Salida,Acceso correcto sin privilegio,NaN
2685,2025-10-21 10:05:47,1.073528e+09,DANIELA,NIEL,NaN,ENTRADA,ENTRADA-1,Huella,ENTRADA-1 Entrada,Apertura con tarjeta de proximidad,NaN


In [5]:
df_prod["nombre_completo"] = df_prod["Nombre"] + " "  + df_prod["Apellido"]
df_prod.head()

,Tiempo,ID de Usuario,Nombre,Apellido,Número de tarjeta,Dispositivo,Punto del evento,Verificación,Estado,Evento,Notas,nombre_completo
0,2025-10-31 17:02:42,1.070920e+08,ARLEY,VANEGAS,NaN,logistica,logistica-1,Huella,Salida,Acceso correcto sin privilegio,NaN,ARLEY VANEGAS
1,2025-10-31 17:01:24,1.057463e+09,YURI NATALIA,GAONA,NaN,logistica,logistica-1,Huella,Salida,Acceso correcto sin privilegio,NaN,YURI NATALIA GAONA
2,2025-10-31 17:00:30,1.031650e+08,LEANDRO,BUSTOS,NaN,logistica,logistica-1,Huella,Salida,Acceso correcto sin privilegio,NaN,LEANDRO BUSTOS
3,2025-10-31 17:00:24,1.014739e+09,ANDRES,ARANGO,NaN,logistica,logistica-1,Huella,Salida,Acceso correcto sin privilegio,NaN,ANDRES ARANGO
4,2025-10-31 16:53:34,8.000000e+00,DANIEL FELIPE,JARAMILLO BUITRAGO,NaN,ENTRADA,ENTRADA-1,Huella,ENTRADA-1 Entrada,Apertura con tarjeta de proximidad,NaN,DANIEL FELIPE JARAMILLO BUITRAGO


### BD info produccion

In [6]:
import xmlrpc.client
import pandas as pd

# ===============================
# Conexión con Odoo
# ===============================
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1) Traer data personal activo 
# ========================================
users = models.execute_kw(
    db, uid, password,
    "hr.employee", "search_read",
    [[ ["active", "=", True]]],
    {"fields": ["department_id","display_name","job_id"]}
)
df = pd.DataFrame(users)

df["department_name"] = df["department_id"].apply(lambda x: x[1] if isinstance(x, list) else None)
df["cargo"] = df["job_id"].apply(lambda x: x[1] if isinstance(x, list) else None)


# Separar la ruta por slash " / "
df[["empresa", "area" ,"trash"]] = df["department_name"].str.split("/", expand=True)

# Limpiar espacios
df["area"] = df["area"].str.strip()
df["cargo"] = df["cargo"].str.strip()

areas = ["PRODUCCION", "MANTENIMIENTO"]



df = df.drop(columns=["trash","id","department_id","job_id","department_name","empresa"])

df_cota =df[df["area"].isin(areas)]

In [7]:
df_cota.sample(10)

,display_name,cargo,area
38,ESTEBAN DAVID DIAZ SEGUANE,OPERARIO DE PLANTA JUNIOR 1,PRODUCCION
113,YENNY KATERINE PATAQUIVA NEUSA,OPERARIO DE PLANTA JUNIOR 1,PRODUCCION
53,JAMER PUPO SIERRA,LIDER DE PROCESO,PRODUCCION
16,CLARA MARLEN OSPINA CASTRO,OPERARIO DE PLANTA JUNIOR 1,PRODUCCION
21,DANILO PUPO SIERRA,OPERARIO DE PLANTA SENIOR,PRODUCCION
1,ALVARO MELO PRADA,OPERARIO DE PLANTA JUNIOR 1,PRODUCCION
35,ELKIN FABIAN DAZA PLAZAS,TECNICO DE MANTENIMIENTO,MANTENIMIENTO
95,ROBERT JULIO CABALLERO MONTEALEGRE,JEFE DE MANTENIMIENTO,MANTENIMIENTO
110,YEISON VILLANUEVA MELO,OPERARIO DE PLANTA JUNIOR 1,PRODUCCION
23,DAVID ESTEBAN MALAGON CETINA,OPERARIO DE PLANTA JUNIOR 2,PRODUCCION


### MATCH TRABAJADORES BIOMETRICO

In [8]:
from fuzzywuzzy import process, fuzz
import pandas as pd
import unidecode

# ---------- 1. Función para normalizar nombre ----------
def limpiar_nombre(nombre):
    if pd.isna(nombre):
        return ""
    s = str(nombre).upper().strip()
    s = unidecode.unidecode(s)      # quita tildes
    s = " ".join(s.split())         # quita espacios dobles
    return s

# ---------- 2. Normalizar columnas ----------
df_prod["nombre_limpio"] = df_prod["nombre_completo"].apply(limpiar_nombre)
df_cota["nombre_limpio"] = df_cota["display_name"].apply(limpiar_nombre)

# Lista única de nombres contra los cuales compararemos
cotas_unique = [n for n in df_cota["nombre_limpio"].unique() if n]

THRESHOLD = 70  # puedes moverlo a 80 si hay nombres muy diferentes

# ---------- 3. Hacer Fuzzy Matching ----------
matches = []  # acumulamos resultados

for idx, row in df_prod.iterrows():
    nombre_prod = row["nombre_limpio"]

    if not nombre_prod:
        matches.append((None, 0))
        continue

    match = process.extractOne(
        nombre_prod,
        cotas_unique,
        scorer=fuzz.token_set_ratio
    )

    if match and match[1] >= THRESHOLD:
        matches.append(match)   # (nombre_en_cota, score)
    else:
        matches.append((None, 0))

df_prod["match_cota"], df_prod["score_match"] = zip(*matches)

# ---------- 4. Merge con df_cota ----------
df_final = df_prod.merge(
    df_cota[["nombre_limpio", "cargo", "area"]],
    left_on="match_cota",
    right_on="nombre_limpio",
    how="left",
    suffixes=("", "_cota")
)

# (opcional) eliminar columnas auxiliares
df_final = df_final.drop(columns=["nombre_limpio_cota"])

# Empleados del registro (df_prod) que NO tuvieron match en df_cota
df_sin_match = df_final[df_final["cargo"].isna()]

# Mostrar nombres únicos sin match
empleados_sin_match = df_sin_match["nombre_completo"].unique()

print("🛑 EMPLEADOS SIN MATCH:")
for e in empleados_sin_match:
    print("-", e)

print("\nCantidad sin match:", len(empleados_sin_match))
df_final = df_final.rename(columns={"Tiempo": "tiempo","area":"Departamento"})




/home/donsson/proyectos/env/lib/python3.12/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/tmp/ipykernel_59383/3852272892.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cota["nombre_limpio"] = df_cota["display_name"].apply(limpiar_nombre)


🛑 EMPLEADOS SIN MATCH:
- ARLEY VANEGAS
- YURI NATALIA GAONA
- LEANDRO BUSTOS
- ANDRES ARANGO
- DANIEL FELIPE JARAMILLO BUITRAGO
- SIOMARA HERNANDEZ
- CRISTIAN DUSSAN
- ADRIANA MARIN
- BRAYAN MUÑOZ
- WILSON DONCEL
- DIEGO MAURICIO MUÑOZ
- DANIELA NIEL
- KATHERIN ANDREA GIL
- nan
- JUAN DAVID GUTIERREZ
- DIDIER BETANCOURT
- JUAN PABLO CANO
- JUAN IGNACIO NIETO NEIVA
- RICARDO ANDRES JUNCA MEDINA
- SERGIO PACHON
- YANID JOTA
- JORGE RACEDO
- JESUS ADONAY GONZALEZ FLOREZ
- ANDREA GH
- GERMAN BETANCOURT
- SIMON B
- MARIANA SANTOS BUITRAGO

Cantidad sin match: 27


In [9]:
print(f"El numero de trabajadores unicos es de ",df_cota['nombre_limpio'].nunique())

El numero de trabajadores unicos es de  32


### HORAS IDEALES DEL MES

In [10]:
import pandas as pd
import holidays

def horas_ideales_a_corte(year, month, fecha_fin_corte):
    """
    Calcula el total de horas ideales de trabajo desde el inicio del mes 
    hasta una fecha de corte específica, excluyendo festivos colombianos.

    :param year: Año del mes a calcular (entero).
    :param month: Mes a calcular (entero).
    :param fecha_fin_corte: Fecha límite para el cálculo (pd.Timestamp o cadena 'YYYY-MM-DD').
    :return: Total de horas ideales redondeado a 2 decimales (flotante).
    """
    # Festivos de Colombia
    # Se genera para el año del inicio del mes y el año de la fecha de corte
    co_holidays = holidays.CountryHoliday("CO", years=[year, fecha_fin_corte.year]) 

    # Rango de días: desde el inicio del mes hasta la fecha de corte
    inicio = pd.Timestamp(year, month, 1)
    # Nos aseguramos que la fecha de corte no sea anterior al inicio del mes
    fin = min(fecha_fin_corte, inicio + pd.offsets.MonthEnd(0)) 
    
    # Asegurarse de que el inicio no sea posterior al fin (en caso de que el mes sea futuro a la fecha de corte)
    if inicio > fin:
        return 0.0

    dias = pd.date_range(inicio, fin, freq="D")

    total_horas = 0.0
    for d in dias:
        if d.date() not in co_holidays:
            if d.weekday() <= 3:  # Lunes a Jueves
                total_horas += 9.5
            elif d.weekday() == 4: # Viernes
                total_horas += 8.5
        # Sábados, domingos y festivos = 0

    return round(total_horas, 2)

# --- EJEMPLO DE USO ---

# Definir los parámetros para el cálculo
año = año
mes = mes
# NOTA: Debes ingresar la fecha de corte como un objeto Timestamp de pandas.
# Si el día de corte es, por ejemplo, el 15 de agosto de 2025:
fecha_corte = pd.Timestamp(año, mes,31) # Puedes cambiar el día (ej: 15)

# Cálculo
horas_del_mes_a_corte = horas_ideales_a_corte(año, mes, fecha_corte)

# Resultado
print(F"\n📅 Calculando Horas Ideales a Corte para el mes {mes}/{año}")
print(F"✂️ Fecha de corte: **{fecha_corte.strftime('%Y-%m-%d')}**")
print(F"🕒 Total de horas ideales esperadas a corte: **{horas_del_mes_a_corte}**")



📅 Calculando Horas Ideales a Corte para el mes 10/2025
✂️ Fecha de corte: **2025-10-31**
🕒 Total de horas ideales esperadas a corte: **204.0**


In [11]:
import pandas as pd
import numpy as np
from datetime import time, date, timedelta
import holidays

# --- Funciones Auxiliares ---

def _is_holiday(d: date, holidays_obj) -> bool:
    """Verifica si una fecha es festivo."""
    if holidays_obj is None:
        return False
    try:
        return d in holidays_obj
    except Exception:
        return d in set(holidays_obj)

def horas_ideales_mes(year=2025, month=8, holidays_obj=None, treat_holidays_as_non_working=True):
    """Calcula las horas ideales para un mes completo."""
    inicio = pd.Timestamp(year, month, 1)
    fin = inicio + pd.offsets.MonthEnd(0)
    dias = pd.date_range(inicio, fin, freq="D").date

    total = 0.0
    for d in dias:
        if d.weekday() <= 3:     # Lun-Jue
            jornada = 9.5
        elif d.weekday() == 4:   # Viernes
            jornada = 8.5
        else:
            jornada = 0.0

        if treat_holidays_as_non_working and _is_holiday(d, holidays_obj) and jornada > 0:
            continue
        total += jornada

    return round(total, 2)

# --- Función Principal Modificada ---

def horas_trabajadas_diario_y_mensual(df, year=2025, month=8,
                                     restar_almuerzo=False, minutos_almuerzo=30,
                                     holidays_obj=None, treat_holidays_as_non_working=True,
                                     debug=False):
    dfx = df.copy()

    if not pd.api.types.is_datetime64_any_dtype(dfx['tiempo']):
        dfx['tiempo'] = pd.to_datetime(dfx['tiempo'], errors='coerce')

    dfx = dfx[dfx['tiempo'].notna()].copy()
    dfx = dfx[(dfx['tiempo'].dt.year == year) & (dfx['tiempo'].dt.month == month)]

    dfx['fecha'] = dfx['tiempo'].dt.date

    agg = (
        dfx.groupby(['nombre_limpio', 'fecha'], as_index=False)
        .agg(
            primera=('tiempo', 'min'),
            ultima=('tiempo', 'max'),
            marcaciones=('tiempo', 'count')
        )
    )

    dept_first = (
        dfx.sort_values(['nombre_limpio', 'fecha', 'tiempo'])
        .groupby(['nombre_limpio', 'fecha'], as_index=False)['Departamento']
        .first()
    )
    agg = agg.merge(dept_first, on=['nombre_limpio', 'fecha'], how='left')

    span_horas = (agg['ultima'] - agg['primera']).dt.total_seconds() / 3600.0
    span_horas = span_horas.where(agg['marcaciones'] >= 2, 0.0)

    if restar_almuerzo:
        span_horas = np.maximum(0.0, span_horas - (minutos_almuerzo / 60.0))

    agg['horas_trabajadas'] = span_horas

    hora_inicio = time(6, 40)
    agg['llegada'] = np.where(
        agg['primera'].isna(),
        "Sin marcación",
        np.where(agg['primera'].dt.time > hora_inicio, "Tarde", "A tiempo")
    )
    
    # 1. CÁLCULO DE HORAS ESPERADAS Y HORAS EXTRA DIARIAS
    
    # Obtener el día de la semana (Lunes=0, Domingo=6)
    agg['weekday'] = pd.to_datetime(agg['fecha']).dt.weekday
    
    # Definir la lógica de horas esperadas por día
    condiciones = [
        agg['weekday'] <= 3,  # Lunes a Jueves: 9.5 horas
        agg['weekday'] == 4   # Viernes: 8.5 horas
    ]
    valores_esperados = [
        9.5,
        8.5
    ]
    
    # Asignar horas esperadas (0.0 para fines de semana por defecto)
    agg['horas_esperadas_dia'] = np.select(condiciones, valores_esperados, default=0.0)
    
    # Aplicar la lógica de festivos
    if treat_holidays_as_non_working and holidays_obj is not None:
        # Identificar días que son festivos Y que NO son sábado/domingo (horas esperadas > 0)
        is_holiday_working_day = agg['fecha'].apply(lambda d: _is_holiday(d, holidays_obj)) & (agg['horas_esperadas_dia'] > 0)
        # Establecer las horas esperadas a 0.0 para esos días
        agg.loc[is_holiday_working_day, 'horas_esperadas_dia'] = 0.0
    
    # 2. CALCULAR HORAS EXTRA
    agg['horas_extra'] = (agg['horas_trabajadas'] - agg['horas_esperadas_dia']).round(2)


    # Diario (reporte con nuevas columnas)
    diario = agg.rename(columns={'nombre_limpio': 'empleado', 'fecha': 'fecha_dia'})[
        ['empleado', 'Departamento', 'fecha_dia', 'marcaciones', 'primera', 'ultima',
         'horas_trabajadas', 'horas_esperadas_dia', 'horas_extra', 'llegada']
    ].copy()

    # Resumen mensual (sin cambios en esta parte, usa la lógica original)
    mensual = (
        diario.groupby(['empleado', 'Departamento'], as_index=False)
        .agg(
            horas_trabajadas_total=('horas_trabajadas', 'sum'),
            horas_extra_total=('horas_extra', 'sum'), # Puedes agregar el total de horas extra aquí
            dias_con_registros=('fecha_dia', 'nunique')
        )
    )
    mensual['horas_trabajadas_total'] = mensual['horas_trabajadas_total'].round(2)
    mensual['horas_extra_total'] = mensual['horas_extra_total'].round(2)

    horas_ideales_const = horas_ideales_mes(year, month, holidays_obj=holidays_obj,
                                             treat_holidays_as_non_working=treat_holidays_as_non_working)
    mensual['horas_ideales'] = horas_ideales_const

    mensual['cumplimiento'] = np.where(
        mensual['horas_trabajadas_total'] >= mensual['horas_ideales'], "Sí", "No"
    )

    mensual = mensual[['empleado', 'Departamento', 'horas_trabajadas_total',
                       'horas_ideales', 'horas_extra_total', 'cumplimiento', 'dias_con_registros']].sort_values('empleado')

    return diario, mensual, horas_ideales_const




In [12]:
diario, mensual, horas_ideales_mes = horas_trabajadas_diario_y_mensual(
    df_final,
    year=año,
    month=mes,
    holidays_obj=co_holidays,           #festivos 🇨🇴
    treat_holidays_as_non_working=True, # True = los festivos no suman horas
    restar_almuerzo=False              # ponlo True si se quiere el tiempo real de trabajo que por ahora no
)


In [13]:
import xlsxwriter

year = año
month = mes

nombre_archivo = f"entregas/mes_produccion_{year}_{month:02d}.xlsx"
with pd.ExcelWriter(nombre_archivo, engine="xlsxwriter") as writer:
    # Guardar las dos hojas
    diario.to_excel(writer, sheet_name="Diario", index=False)
    mensual.to_excel(writer, sheet_name="Mensual", index=False)

    # === AQUI EMPIEZA EL FORMATO CONDICIONAL ===
    workbook = writer.book
    ws_mensual = writer.sheets["Mensual"]


    # Columnas clave
    col_cumplimiento = mensual.columns.get_loc("cumplimiento") + 1  # +1 por índice Excel
    col_dias = mensual.columns.get_loc("dias_con_registros") + 1

    ultima_fila = len(mensual) + 1  # +1 por el encabezado